# OCR Matching Algorithm Results & Benchmarks

This notebook is to document the performance of the matching algorithm which checks the values returned by OCR against the voter registry dataframe. It will mainly serve to illustrate the results along two main axes, the search function and the scorer used by the fuzzy matching function.

In [1]:
# libraries used for matching
import pandas as pd
import numpy as np
from rapidfuzz import fuzz, process, utils
import time
from loguru  import logger
import sys
import json
import glob

# creating dataframe from the registry - this file must be requested for use
voter_records_2023_df = pd.read_csv('../data/raw_feb_23_city_wide.csv', dtype=str)

# removing default sink and adding Jupyter Notebook system standout as a sink for logger
logger.remove()
logger.add(sys.stdout, level="INFO", colorize=True, format="<r><u>{time}</u></r> <green>{level}</green> <black><bold>{message}</bold></black>")

# loading the OCR Results - this file must be requested for use
with open('../data/processed_ocr_data.json', 'r') as file:
    resulting_data = json.load(file)

## Vanilla

The original version of the full name column generation implements a lambda function, we then create a separate list of full name values, and the matching function iterates through the full name list to check every name returned by OCR.

In [2]:
# Creating Full Name column with lambda
start = time.time()
voter_records_2023_df['Full Name'] = voter_records_2023_df.apply(lambda x: f"{x['First_Name']} {x['Last_Name']}", axis=1)
full_name_list = list(voter_records_2023_df['Full Name'])
end = time.time()
logger.info(f"Columns generated in {end - start:.2f} seconds")

2024-08-18T10:44:47.324344-0400 INFO Columns generated in 3.94 seconds


In [3]:
##
# FUZZY MATCHING FUNCTION
##

def score_function_fuzz(ocr_name, full_name_list):

    """
    Outputs the voter record indices of the names that are
    closest to `ocr_name`.
    """

    # empty dictionary of scores
    full_name_score_dict = dict()

    for idx in range(len(full_name_list)):

        # getting full name for row; ensuring string
        name_row = str(full_name_list[idx])

        # converting string to lower case to simplify matching
        name_row = name_row.lower()
        ocr_name = ocr_name.lower()

        # compiling scores; writing as between 0 and 1
        full_name_score_dict[idx] = fuzz.ratio(ocr_name, name_row)/100

    # sorting dictionary
    sorted_dictionary = dict(sorted(full_name_score_dict.items(), reverse=True, key=lambda item: item[1]))

    # top five key value pairs (indices and scores)
    indices_scores_list = list(sorted_dictionary.items())[:5]

    return indices_scores_list

### By Full Name

In [4]:
matched_list = list()
start_time = time.time()

for dict_ in resulting_data:
                temp_dict = dict()
                high_match_ids = score_function_fuzz(dict_['Name'], full_name_list)
                id_, score_ = high_match_ids[0]
                temp_dict['OCR NAME'] = str(dict_['Name'])
                temp_dict['MATCHED NAME'] = full_name_list[id_]
                temp_dict['SCORE'] = "{:.2f}".format(score_)
                temp_dict['VALID'] = False
                if score_ > 0.85:
                    temp_dict['VALID'] = True
                matched_list.append(temp_dict)

match_df = pd.DataFrame(matched_list, columns=["OCR NAME", "MATCHED NAME", "SCORE", "VALID"])
end_time = time.time()
match_df

,OCR NAME,MATCHED NAME,SCORE,VALID
0,Mario L. Hoyer,Mario Glover,0.77,False
1,Carlos R. Guillen,Carlos Guillen,0.90,True
2,Derek Hower,Derek Bower,0.91,True
3,Jacqueline Chapman,Jacqueline Chapman,1.00,True
4,Christopher Dick,Christopher Dieck,0.97,True
...,...,...,...,...
61,Caroline Ritter,Caroline Ritter,1.00,True
62,John Doe,John Doble,0.89,True
63,Jane Plane,Jan Lane,0.89,True
64,Kermit Turner,Kermit Turner,1.00,True


In [5]:
valid_matches = match_df['VALID'].sum()
total_records = len(match_df)
logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T10:45:54.238739-0400 INFO Match Time 66.880 secs | Matched Records: 53 of 66 - 80.30%


## Vectorized Columns

For greater specificity when matching voter records, we wish to include the address of the voter as well as their name in the search. Using vectorized columns in lieu of the lambda function speeds up the column generation and we can omit the list as extract can iterate through a panda series directly.



In [6]:
start_time = time.time()
voter_records_2023_df["Full Name"] = voter_records_2023_df["First_Name"] + ' ' + voter_records_2023_df["Last_Name"]
voter_records_2023_df["Full Address"] =  voter_records_2023_df["Street_Number"] + " " + voter_records_2023_df["Street_Name"] + " " + voter_records_2023_df["Street_Type"] + " " + voter_records_2023_df["Street_Dir_Suffix"]
voter_records_2023_df["Full Name and Full Address"] = voter_records_2023_df["Full Name"] + ' ' + voter_records_2023_df["Full Address"]
end_time = time.time()

logger.info(f"Initialized columns in {end_time - start_time:.2f} seconds")

2024-08-18T10:45:55.105201-0400 INFO Initialized columns in 0.86 seconds


## Extract

Extract simplifies the matching process significantly, we simply feed it a query and a list of values to check the query against, and it returns a list of tuples which range from the highest match score to the lowest.

In [7]:
def score_extract_default(query_name, names_list):
    # the default scorer produces a Levenshtein distance number, so we can use fuzz.ratio as a scorer to obtain a more readable % format
    # default_process is a processor that removes whitespace, lowers all letters, removes any non-alphanumeric characters
    # by limiting to 5 we can save a little bit of space and time and code, since we were looping through all of the items before and only taking the top 5 of the sorted list.
    list_of_match_tuples = process.extract(query=query_name, choices=names_list, scorer=fuzz.ratio, processor=utils.default_process, limit=5)
    # this will produce a list of tuples whose values are as follows:
    # (matched record: the record which the query matched*,
    # score: the result of fuzz.ratio between the query and the matched record,
    # index: when checked against an iterable i.e standard python list, this will be an index, when checked against panda dataframes, it will return a key)
    return list_of_match_tuples

### By Name + Address

When we receive data back from the OCR, it is in the form of a dictionary with keys for Name, Address, Ward, and Date. We combine the values for Name and Address to check against the OCR column we created earlier. It's important to note that with the % format returned by our new scorer we have to change the valid score check from 0.85 to 85.0.

In [8]:
matched_list = list()
start_time = time.time()

for dict_ in resulting_data:
                name_address_combo = f"{dict_['Name']} {dict_['Address']}"
                temp_dict = dict()
                high_match_ids = score_extract_default(name_address_combo, voter_records_2023_df["Full Name and Full Address"])
                record_, score_, id_ = high_match_ids[0]
                temp_dict['OCR RECORD'] = name_address_combo
                temp_dict['MATCHED RECORD'] = record_
                temp_dict['SCORE'] = "{:.2f}".format(score_)
                temp_dict['VALID'] = False
                if score_ > 85.0:
                    temp_dict['VALID'] = True
                matched_list.append(temp_dict)

match_df = pd.DataFrame(matched_list, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
end_time = time.time()
match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mark Flowers 802 Decatur St NW,79.37,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,92.75,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,94.74,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman 4726 5th ST NW,75.76,False
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,98.41,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,John Howe 4400 50th St NW,82.61,False
63,Jane Plane 300 Good Hope Rd SE,Janice Sanker 2300 Good Hope Rd SE,87.50,True
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,89.66,True


In [9]:
valid_matches = match_df['VALID'].sum()
total_records = len(resulting_data)
logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T10:46:12.601788-0400 INFO Match Time 17.474 secs | Matched Records: 32 of 66 - 48.48%


### By Ward

We are now searching by name and address, however combing through the entire registry dataframe is more time intensive than we'd like. As we have access to the Ward value in the data returned by OCR, we can search name an address only within the Ward and cut the search time considerably. Because the WARD value in the voter registry is entered as a float, we have to do a little string formatting.

In [10]:
matched_list = list()
start_time = time.time()

for dict_ in resulting_data:
                name_address_combo = f"{dict_['Name']} {dict_['Address']}"
                temp_dict = dict()
                high_match_ids = score_extract_default(name_address_combo, voter_records_2023_df[voter_records_2023_df['WARD'] == f"{dict_['Ward']}.0"]["Full Name and Full Address"])
                record_, score_, id_ = high_match_ids[0]
                temp_dict['OCR RECORD'] = name_address_combo
                temp_dict['MATCHED RECORD'] = record_
                temp_dict['SCORE'] = "{:.2f}".format(score_)
                temp_dict['VALID'] = False
                if score_ > 85.0:
                    temp_dict['VALID'] = True
                matched_list.append(temp_dict)

match_df = pd.DataFrame(matched_list, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
end_time = time.time()
match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mark Flowers 802 Decatur St NW,79.37,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,92.75,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,94.74,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman 4726 5th ST NW,75.76,False
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,98.41,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,John Boyle 1440 R St NW,81.82,False
63,Jane Plane 300 Good Hope Rd SE,Janice Sanker 2300 Good Hope Rd SE,87.50,True
64,Kermit Turner 2333 15th St NE,Terri Turner 2725 13th St NE,77.19,False


In [11]:
valid_matches = match_df['VALID'].sum()
total_records = len(match_df)
logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T10:46:20.967277-0400 INFO Match Time 8.348 secs | Matched Records: 26 of 66 - 39.39%


## Hierarchical Search

At this point we can match by full name + address, as well as by Ward. OCR is imperfect, and has a tendency to return incorrect or entirely hallucinated Ward values. This means that searching by Ward is not always optimal, but if we can find a valid match within the Ward returned by OCR we'd like to keep it without having to search further. By using a hierarchical search we can split the difference between speed and accuracy. If a valid match is found in ward, we return it, and if not we move on to a name + address search against the entire registry. If no match is found for name + address in the registry, we default to the original Full Name search.

The first step in implementing this search is to refactor the extract function, so we can use different scores for different searches, and to implement adjustable limits to the returns. The fuzz.token_ratio scorer seems to perform better when matching addresses, and fuzz.ratio seems to perform better matching names, however not every combination of the scorers offered by rapidfuzz has been tested so far.

In [12]:
def score_extract(ocr_name, full_name_list, scorer_=fuzz.token_ratio, limit_=1):
    list_of_match_tuples = process.extract(query=ocr_name, choices=full_name_list, scorer=scorer_, processor=utils.default_process, limit=limit_)
    return list_of_match_tuples

Given the length of the function, it makes sense to abstract the search into its own function to preserve legibility.

In [13]:
def tiered_search(name, address):

    name_address_combo = f"{name} {address}"

    # Searches for a match within the Ward returned by OCR
    name_address_matches1 = score_extract(name_address_combo, voter_records_2023_df[voter_records_2023_df['WARD'] == f"{dict_['Ward']}.0"]["Full Name and Full Address"])
    name_address__name1, name_address__score1, name_address__id1 = name_address_matches1[0]

    # if score is more than 85, return the tuple
    if name_address__score1 >= 85.0:
        return (name_address__name1, name_address__score1, name_address__id1)

    # if score is below 85, do additional processing
    else:

        # computing matches based on name and address; only considers all other wards
        name_address_matches2 = score_extract(name_address_combo, voter_records_2023_df[voter_records_2023_df['WARD'] != f"{dict_['Ward']}.0"]["Full Name and Full Address"])
        name_address__name2, name_address__score2, name_address__id2 = name_address_matches2[0]

        # if the new voter records score is greater than 85, return tuple
        if name_address__score2 >= 85.0:
            return (name_address__name2, name_address__score2, name_address__id2)

        # if score is less than 85, perform full records search based on name
        # and return results with highest score
        else:
            # computing matches based on name alone; considers full voter records
            full_name_matches = score_extract(name, voter_records_2023_df["Full Name"], scorer_=fuzz.ratio)
            full_name__name, full_name__score, full_name__id = full_name_matches[0]

            # find max from three scores
            max_indx = np.argmax([name_address__score1, name_address__score2, full_name__score])

            # return records associated with that max
            if max_indx== 0:
                return (name_address__name1, name_address__score1, name_address__id1)
            elif max_indx == 1:
                return (name_address__name2, name_address__score2, name_address__id2)
            else:
                address = voter_records_2023_df.loc[full_name__id, 'Full Address']
                full_name_address = f"{full_name__name} {address}"
                return (full_name_address, full_name__score, full_name__id)

In [14]:
matched_list = list()
start_time = time.time()

for dict_ in resulting_data:
    temp_dict = dict()
    name_, score_, id_ = tiered_search(dict_['Name'], dict_['Address'])
    temp_dict['OCR RECORD'] = f"{dict_['Name']} {dict_['Address']}"
    temp_dict['MATCHED RECORD'] = name_
    temp_dict['SCORE'] = "{:.2f}".format(score_)
    temp_dict['VALID'] = False
    if score_ > 85.0:
        temp_dict['VALID'] = True
    matched_list.append(temp_dict)

## Editable Table
match_df = pd.DataFrame(matched_list, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
end_time = time.time()
match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Dale Learn 1202 Decatur ST NW,78.69,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,94.12,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,94.74,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman 4726 5th ST NW,100.00,True
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,98.41,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,John Doble 2400 41st St NW,88.89,True
63,Jane Plane 300 Good Hope Rd SE,Janice Sanker 2300 Good Hope Rd SE,87.50,True
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,89.66,True


In [15]:
total_records = len(match_df)
valid_matches = match_df["VALID"].sum()
logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T10:48:11.242841-0400 INFO Match Time 110.241 secs | Matched Records: 59 of 66 - 89.39%


# Benchmarking Scorers

Below are benchmarks of all scorers available through rapidfuzz for name + address search and full name search. I've commented out all the actual matching and tables so that this notebook doesn't take 10 minutes to run, while keeping the values returned. The match time is system dependent but should provide a general idea of how fast the scorers are in relation to each other. Sadly visualizing this data is difficult because the most promising results usually include the highest rate of false positives.

 Cursory analysis would suggest fuzz.ratio as the best scorer for the Full Name search and either fuzz.token_set_ratio (accuracy) or token_sort_ratio (speed, fewer false positives) as the best scorer for Full Name + Full Address search. 

First, let's create a function to simplify running the loop for test data.

In [27]:
def benchmark_scorer(record_list, scorer=fuzz.WRatio, name_address=False):
    matched_list = list()
    if name_address:
        df = voter_records_2023_df["Full Name and Full Address"]
    else:
        df = voter_records_2023_df["Full Name"]
        
    for dict_ in record_list:
        temp_dict = dict()
        if name_address:
            record_name_address = f"{dict_['Name']} {dict_['Address']}"
            matched_records = score_extract(record_name_address, df, scorer_=scorer)
            name_, score_, id_ = matched_records[0]
        else:
            matched_records = score_extract(dict_['Name'], df, scorer_=scorer)
            name_, score_, id_ = matched_records[0]
        temp_dict['OCR RECORD'] = f"{dict_['Name']} {dict_['Address']}"
        temp_dict['MATCHED RECORD'] = name_
        temp_dict['SCORE'] = "{:.2f}".format(score_)
        temp_dict['VALID'] = False
        if score_ > 85.0:
            temp_dict['VALID'] = True
        matched_list.append(temp_dict)
    
    ## Editable Table
    return matched_list

## fuzz.WRatio
### Name and Address

Match time:  140s
Valid match rate: 100.00%
Notes: Returns obvious false positives - all records seem to have a match score floor of 85.50. Without further research this is unusable for our purposes.

In [28]:
# start_time = time.time()
# name_address_results__WRatio = benchmark_scorer(resulting_data, name_address=True)
# match_df = pd.DataFrame(name_address_results__WRatio, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Hamed Aali 254 M ST SW,85.50,True
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,92.75,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,94.74,True
3,Jacqueline Chapman Channing St NE,Hamed Aali 254 M ST SW,85.50,True
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,98.41,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,Rebecca A Blessing 1722 19th ST NW,85.50,True
63,Jane Plane 300 Good Hope Rd SE,Janice Sanker 2300 Good Hope Rd SE,87.50,True
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,89.66,True


In [29]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:09:56.358322-0400 INFO Match Time 139.783 secs | Matched Records: 66 of 66 - 100.00%


### Full Name

Match time: 64s
Valid match rate: 98.48%
Notes: Unusable for current search. Wratio returns fewer false positives when used in the full name check, but only because it seems to find matches more consistently. Where there are no good matches, it still returns obvious mismatches with an 85.5 match rating.

In [30]:
# start_time = time.time()
# name_address_results__WRatio = benchmark_scorer(resulting_data)
# match_df = pd.DataFrame(name_address_results__WRatio, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,L Best,85.50,True
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen,95.00,True
2,Derek Hower 836 Varnum St NW,Derek Bower,90.91,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman,100.00,True
4,Christopher Dick 4726 5th St NW,Christopher Dieck,96.97,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter,100.00,True
62,John Doe 4400 A St NW,John Doble,88.89,True
63,Jane Plane 300 Good Hope Rd SE,Jan Lane,88.89,True
64,Kermit Turner 2333 15th St NE,Kermit Turner,100.00,True


In [31]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:11:00.441732-0400 INFO Match Time 64.067 secs | Matched Records: 65 of 66 - 98.48%


## fuzz.ratio

### Name & Address
Match time: 14.3s
Match Rate: 48.48%

In [32]:
# start_time = time.time()
# name_address_results__r = benchmark_scorer(resulting_data, scorer=fuzz.ratio, name_address=True)
# match_df = pd.DataFrame(name_address_results__r, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mark Flowers 802 Decatur St NW,79.37,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,92.75,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,94.74,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman 4726 5th ST NW,75.76,False
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,98.41,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,John Howe 4400 50th St NW,82.61,False
63,Jane Plane 300 Good Hope Rd SE,Janice Sanker 2300 Good Hope Rd SE,87.50,True
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,89.66,True


In [33]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:19:29.022056-0400 INFO Match Time 14.279 secs | Matched Records: 32 of 66 - 48.48%


### Full Name

Match time: 11.327s
Match rate: 80.3%
Notes: This is the most consistently performant and accurate scorer for the full name check.

In [34]:
# start_time = time.time()
# full_name_results__r = benchmark_scorer(resulting_data, scorer=fuzz.ratio)
# match_df = pd.DataFrame(full_name_results__r, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mario Glover,76.92,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen,90.32,True
2,Derek Hower 836 Varnum St NW,Derek Bower,90.91,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman,100.00,True
4,Christopher Dick 4726 5th St NW,Christopher Dieck,96.97,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter,100.00,True
62,John Doe 4400 A St NW,John Doble,88.89,True
63,Jane Plane 300 Good Hope Rd SE,Jan Lane,88.89,True
64,Kermit Turner 2333 15th St NE,Kermit Turner,100.00,True


In [35]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:19:40.367931-0400 INFO Match Time 11.327 secs | Matched Records: 53 of 66 - 80.30%


## fuzz.partial_ratio

### Name & Address

Match time: 77.635s
Match rate: 51.52s

In [36]:
# start_time = time.time()
# name_address_results__pr = benchmark_scorer(resulting_data, scorer=fuzz.partial_ratio, name_address=True)
# match_df = pd.DataFrame(name_address_results__pr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Dale Learn 1202 Decatur ST NW,84.00,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,87.88,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,92.86,True
3,Jacqueline Chapman Channing St NE,Amy Chang 16 Channing St NE,80.85,False
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,96.77,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,Alexia Arredondo 440 K ST NW,81.08,False
63,Jane Plane 300 Good Hope Rd SE,Renee Abney 2300 Good Hope Rd SE,86.67,True
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,91.23,True


In [37]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:20:58.018647-0400 INFO Match Time 77.635 secs | Matched Records: 34 of 66 - 51.52%


### Full Name

Match time: 32.60s
Match rate: 85.45
Notes: Unusuable for rate of false positives.

In [38]:
# start_time = time.time()
# full_name_results__pr = benchmark_scorer(resulting_data, scorer=fuzz.partial_ratio)
# match_df = pd.DataFrame(full_name_results__pr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,J Hyer,83.33,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Cruz,84.21,False
2,Derek Hower 836 Varnum St NW,Derek Ho,100.00,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman,100.00,True
4,Christopher Dick 4726 5th St NW,Christopher Dicken,100.00,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter,100.00,True
62,John Doe 4400 A St NW,John De Lancie,93.33,True
63,Jane Plane 300 Good Hope Rd SE,Jayne Plank,90.00,True
64,Kermit Turner 2333 15th St NE,T T,100.00,True


In [39]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:21:30.636732-0400 INFO Match Time 32.601 secs | Matched Records: 63 of 66 - 95.45%


## fuzz.token_set_ratio

### Name & Address

Match time: 122.430s
Match rate: 57.58%
Notes:  A very accurate scorer with a low false positive rate in the OCR data set, but slow.

In [40]:
# start_time = time.time()
# name_address_results__psetr = benchmark_scorer(resulting_data, scorer=fuzz.token_set_ratio, name_address=True)
# match_df = pd.DataFrame(name_address_results__psetr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Dale Learn 1202 Decatur ST NW,78.69,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,94.12,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,94.74,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman 4726 5th ST NW,77.78,False
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,98.41,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,John Howe 4400 50th St NW,83.33,False
63,Jane Plane 300 Good Hope Rd SE,Janice Sanker 2300 Good Hope Rd SE,87.50,True
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,89.66,True


In [41]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:23:33.082705-0400 INFO Match Time 122.430 secs | Matched Records: 38 of 66 - 57.58%


### Full Name
Match time: 75.748s
Match rate: 51.52%

In [42]:
# start_time = time.time()
# full_name_results__psetr = benchmark_scorer(resulting_data, scorer=fuzz.token_set_ratio)
# match_df = pd.DataFrame(full_name_results__psetr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Dale Learn 1202 Decatur ST NW,84.00,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,87.88,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,92.86,True
3,Jacqueline Chapman Channing St NE,Amy Chang 16 Channing St NE,80.85,False
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,96.77,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,Alexia Arredondo 440 K ST NW,81.08,False
63,Jane Plane 300 Good Hope Rd SE,Renee Abney 2300 Good Hope Rd SE,86.67,True
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,91.23,True


In [43]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:24:51.847738-0400 INFO Match Time 78.748 secs | Matched Records: 34 of 66 - 51.52%


## fuzz.partial_token_set_ratio

### Name & Address
Match time: .003s
Match rate: 100%
Notes: Returns first record in the database for nearly every query, unusuable.

In [44]:
# start_time = time.time()
# name_address_results__ptsetr = benchmark_scorer(resulting_data, scorer=fuzz.partial_token_set_ratio, name_address=True)
# match_df = pd.DataFrame(name_address_results__ptsetr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
1,Carlos R. Guillen 980 Randolph St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
2,Derek Hower 836 Varnum St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
3,Jacqueline Chapman Channing St NE,Rebecca A Blessing 1722 19th ST NW,100.00,True
4,Christopher Dick 4726 5th St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
62,John Doe 4400 A St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
63,Jane Plane 300 Good Hope Rd SE,Erik Aaby 1300 4th St SE,100.00,True
64,Kermit Turner 2333 15th St NE,Rebecca A Blessing 1722 19th ST NW,100.00,True


In [45]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:24:51.869474-0400 INFO Match Time 0.003 secs | Matched Records: 66 of 66 - 100.00%


### Full Name

Match time: 1.766s
Match Rate: 100%
Notes: Unusable/False positives.

In [46]:
# start_time = time.time()
# full_name_results__ptsetr = benchmark_scorer(resulting_data, scorer=fuzz.partial_token_set_ratio)
# match_df = pd.DataFrame(full_name_results__ptsetr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mario Acosta-Velez,100.00,True
1,Carlos R. Guillen 980 Randolph St NW,Carlos Abreu,100.00,True
2,Derek Hower 836 Varnum St NW,Derek Aalerud,100.00,True
3,Jacqueline Chapman Channing St NE,Jacqueline Abbott,100.00,True
4,Christopher Dick 4726 5th St NW,Christopher Abad,100.00,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ackerman,100.00,True
62,John Doe 4400 A St NW,John Abbey,100.00,True
63,Jane Plane 300 Good Hope Rd SE,Mary Jane Abordo,100.00,True
64,Kermit Turner 2333 15th St NE,Turner Anna Leigh,100.00,True


In [47]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:24:53.652087-0400 INFO Match Time 1.766 secs | Matched Records: 66 of 66 - 100.00%


## fuzz.token_sort_ratio

### Name & Address

Match time: 58.513
Match Rate: 46.97%
Notes: This is far more performant and does not return many false positives, which may be why it's match rate is so low. I would recommend it for use. The match rate would suffer but probably would better represent the validity of the match.

In [48]:
# start_time = time.time()
# name_address_results__tsr = benchmark_scorer(resulting_data, scorer=fuzz.token_sort_ratio, name_address=True)
# match_df = pd.DataFrame(name_address_results__tsr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mark Flowers 802 Decatur St NW,77.42,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,94.12,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,94.74,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman 4726 5th ST NW,75.76,False
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,98.41,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,John Howe 4400 50th St NW,82.61,False
63,Jane Plane 300 Good Hope Rd SE,Jaja Parker 2300 Good Hope Rd SE,83.87,False
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,89.66,True


In [49]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:25:52.181095-0400 INFO Match Time 58.513 secs | Matched Records: 31 of 66 - 46.97%


### Full Name
Match time: 25.767s
Match rate: 80.30%
Notes: A solid blend of performance and accuracy, but ratio is still probably the better scorer.

In [50]:
# start_time = time.time()
# full_name_results__tsr = benchmark_scorer(resulting_data, scorer=fuzz.token_sort_ratio)
# match_df = pd.DataFrame(full_name_results__tsr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mario Ayele,75.00,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen,93.33,True
2,Derek Hower 836 Varnum St NW,Derek Howard,86.96,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman,100.00,True
4,Christopher Dick 4726 5th St NW,Christopher Dieck,96.97,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter,100.00,True
62,John Doe 4400 A St NW,John Doble,88.89,True
63,Jane Plane 300 Good Hope Rd SE,Jan Lane,88.89,True
64,Kermit Turner 2333 15th St NE,Kermit Turner,100.00,True


In [51]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:26:17.963874-0400 INFO Match Time 25.767 secs | Matched Records: 53 of 66 - 80.30%


## fuzz.partial_token_sort_ratio

### Name & Address
Match time: 124.710s
Match rate: 54.55%
Notes: One of the slowest scorers, with some obvious false positives. It is generally accurate, but 

In [52]:
# start_time = time.time()
# name_address_results__ptsr = benchmark_scorer(resulting_data, scorer=fuzz.partial_token_sort_ratio, name_address=True)
# match_df = pd.DataFrame(name_address_results__ptsr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mark Flowers 802 Decatur St NW,80.00,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,93.94,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,92.86,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman 4726 5th ST NW,84.75,False
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,96.77,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,John Adlon 3060 16th St NW,82.93,False
63,Jane Plane 300 Good Hope Rd SE,Jatan Tipps 2300 Good Hope Rd SE,85.71,True
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,89.66,True


In [53]:
total_records = len(match_df)
valid_matches = match_df["VALID"].sum()
logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:28:22.688703-0400 INFO Match Time 124.710 secs | Matched Records: 36 of 66 - 54.55%


### Full Name
Match time: 54.77s
Match rate: 96.97%
Notes: Unusable/false positives.

In [54]:
# start_time = time.time()
# full_name_results__ptsr = benchmark_scorer(resulting_data, scorer=fuzz.partial_token_sort_ratio)
# match_df = pd.DataFrame(full_name_results__ptsr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,J Hyer,83.33,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen,100.00,True
2,Derek Hower 836 Varnum St NW,Derek Ho,100.00,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman,100.00,True
4,Christopher Dick 4726 5th St NW,Christopher Dicken,100.00,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter,100.00,True
62,John Doe 4400 A St NW,John Bledsoe,93.33,True
63,Jane Plane 300 Good Hope Rd SE,Jayne Plank,90.00,True
64,Kermit Turner 2333 15th St NE,T T,100.00,True


In [55]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:29:17.476793-0400 INFO Match Time 54.773 secs | Matched Records: 64 of 66 - 96.97%


## fuzz.token_ratio

### Name & Address
Match time: 144.637s
Match rate: 57.58%
Notes: Slow, probably a bit too strict on some matching, but strong accuracy.

In [57]:
# start_time = time.time()
# name_address_results__tr = benchmark_scorer(resulting_data, scorer=fuzz.token_ratio, name_address=True)
# match_df = pd.DataFrame(name_address_results__tr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Dale Learn 1202 Decatur ST NW,78.69,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen 960 Randolph ST NW,94.12,True
2,Derek Hower 836 Varnum St NW,Derek Howell 836 Varnum St NW,94.74,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman 4726 5th ST NW,77.78,False
4,Christopher Dick 4726 5th St NW,Christopher Dieck 4726 5th St NW,98.41,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter 3901 Connecticut Ave NW,100.00,True
62,John Doe 4400 A St NW,John Howe 4400 50th St NW,83.33,False
63,Jane Plane 300 Good Hope Rd SE,Janice Sanker 2300 Good Hope Rd SE,87.50,True
64,Kermit Turner 2333 15th St NE,Kermit Turner 2325 15th St NW,89.66,True


In [58]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:37:48.147838-0400 INFO Match Time 143.637 secs | Matched Records: 38 of 66 - 57.58%


### Full Name
Match time: 59.102s
Match rate: 83.33%

In [59]:
# start_time = time.time()
# full_name_results__tr = benchmark_scorer(resulting_data, scorer=fuzz.token_ratio)
# match_df = pd.DataFrame(full_name_results__tr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mario Ayele,75.00,False
1,Carlos R. Guillen 980 Randolph St NW,Carlos Guillen,100.00,True
2,Derek Hower 836 Varnum St NW,Derek Bower,90.91,True
3,Jacqueline Chapman Channing St NE,Jacqueline Chapman,100.00,True
4,Christopher Dick 4726 5th St NW,Christopher Dieck,96.97,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ritter,100.00,True
62,John Doe 4400 A St NW,John Doble,88.89,True
63,Jane Plane 300 Good Hope Rd SE,Jan Lane,88.89,True
64,Kermit Turner 2333 15th St NE,Kermit Turner,100.00,True


In [60]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:38:47.265875-0400 INFO Match Time 59.102 secs | Matched Records: 55 of 66 - 83.33%


## fuzz.partial_token_ratio

### Name & Address
Match time: .002s
Match rate: 100%
Notes: Unusable/false positives.

In [61]:
# start_time = time.time()
# name_address_results__ptr = benchmark_scorer(resulting_data, scorer=fuzz.partial_token_ratio, name_address=True)
# match_df = pd.DataFrame(name_address_results__ptr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
1,Carlos R. Guillen 980 Randolph St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
2,Derek Hower 836 Varnum St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
3,Jacqueline Chapman Channing St NE,Rebecca A Blessing 1722 19th ST NW,100.00,True
4,Christopher Dick 4726 5th St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
62,John Doe 4400 A St NW,Rebecca A Blessing 1722 19th ST NW,100.00,True
63,Jane Plane 300 Good Hope Rd SE,Erik Aaby 1300 4th St SE,100.00,True
64,Kermit Turner 2333 15th St NE,Rebecca A Blessing 1722 19th ST NW,100.00,True


In [62]:
# total_records = len(match_df)
# valid_matches = match_df["VALID"].sum()
# logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:38:47.287365-0400 INFO Match Time 0.002 secs | Matched Records: 66 of 66 - 100.00%


### Full Name
Match time: 1.6s
Match rate: 100%
Notes: Unusable/False Postives.

In [63]:
# start_time = time.time()
# full_name_results__ptr = benchmark_scorer(resulting_data, scorer=fuzz.partial_token_ratio)
# match_df = pd.DataFrame(full_name_results__ptr, columns=["OCR RECORD", "MATCHED RECORD", "SCORE", "VALID"])
# end_time = time.time()
# match_df

,OCR RECORD,MATCHED RECORD,SCORE,VALID
0,Mario L. Hoyer 1202 Decatur St NW,Mario Acosta-Velez,100.00,True
1,Carlos R. Guillen 980 Randolph St NW,Carlos Abreu,100.00,True
2,Derek Hower 836 Varnum St NW,Derek Aalerud,100.00,True
3,Jacqueline Chapman Channing St NE,Jacqueline Abbott,100.00,True
4,Christopher Dick 4726 5th St NW,Christopher Abad,100.00,True
...,...,...,...,...
61,Caroline Ritter 3901 Connecticut Ave NW,Caroline Ackerman,100.00,True
62,John Doe 4400 A St NW,John Abbey,100.00,True
63,Jane Plane 300 Good Hope Rd SE,Mary Jane Abordo,100.00,True
64,Kermit Turner 2333 15th St NE,Turner Anna Leigh,100.00,True


In [64]:
total_records = len(match_df)
valid_matches = match_df["VALID"].sum()
logger.info(f"Match Time {end_time-start_time:.3f} secs | Matched Records: {valid_matches} of {total_records} - {(valid_matches/total_records * 100):.2f}%")

2024-08-18T11:38:48.983747-0400 INFO Match Time 1.682 secs | Matched Records: 66 of 66 - 100.00%
